In [11]:
#PLANET_API_KEY='cfb9acbc5bdb4cb496bcf33d57d28738'
from planet import api
from requests.auth import HTTPBasicAuth
from osgeo import ogr, gdal, osr
import xml.dom.minidom
import http.client

import math


import requests, json, geopandas as gpd
import os
import pandas as pd
import geojson
import numpy as np
from pygeotile.tile import Tile
from pyproj import Proj, transform

from PIL import Image

from pymongo import MongoClient 
import datetime
clientMongo = MongoClient('localhost', 27017) #Cliente mongo para conexion a repositorio de tiles y chips
db = clientMongo.deepAlerts

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
clientPlanet = api.ClientV1()

In [12]:
result = requests.get('https://api.planet.com/basemaps/v1/mosaics', auth=HTTPBasicAuth(os.environ['PL_API_KEY_IDEAM'], ''))
result = json.loads(result.text)
#PLANET_API_KEY
#'PL_API_KEY_IDEAM'
#PL_API_KEY

In [13]:
result

{'_links': {'_self': 'https://api.planet.com/basemaps/v1/mosaics?api_key=5a78a5a455644480a8a40ce5b4af9caa'},
 'mosaics': [{'_links': {'_self': 'https://api.planet.com/basemaps/v1/mosaics/48fff803-4104-49bc-b913-7467b7a5ffb5?api_key=5a78a5a455644480a8a40ce5b4af9caa',
    'tiles': 'https://tiles.planet.com/basemaps/v1/planet-tiles/global_monthly_2016_01_mosaic/gmap/{z}/{x}/{y}.png?api_key=5a78a5a455644480a8a40ce5b4af9caa'},
   'bbox': [-179, -56, 180, 58],
   'coordinate_system': 'EPSG:3857',
   'datatype': 'byte',
   'first_acquired': '2016-01-01T00:00:00.000Z',
   'grid': {'quad_size': 2048, 'resolution': 4.777314267823516},
   'id': '48fff803-4104-49bc-b913-7467b7a5ffb5',
   'interval': '1 mon',
   'item_types': ['PSScene3Band', 'REOrthoTile'],
   'last_acquired': '2016-02-01T00:00:00.000Z',
   'level': 15,
   'name': 'global_monthly_2016_01_mosaic',
   'product_type': 'timelapse'},
  {'_links': {'_self': 'https://api.planet.com/basemaps/v1/mosaics/f2952136-69bc-4c31-bc21-a2e3919042a1

In [14]:
x=list(filter(lambda person: '201' in person['name'], result['mosaics']))
x

[{'_links': {'_self': 'https://api.planet.com/basemaps/v1/mosaics/48fff803-4104-49bc-b913-7467b7a5ffb5?api_key=5a78a5a455644480a8a40ce5b4af9caa',
   'tiles': 'https://tiles.planet.com/basemaps/v1/planet-tiles/global_monthly_2016_01_mosaic/gmap/{z}/{x}/{y}.png?api_key=5a78a5a455644480a8a40ce5b4af9caa'},
  'bbox': [-179, -56, 180, 58],
  'coordinate_system': 'EPSG:3857',
  'datatype': 'byte',
  'first_acquired': '2016-01-01T00:00:00.000Z',
  'grid': {'quad_size': 2048, 'resolution': 4.777314267823516},
  'id': '48fff803-4104-49bc-b913-7467b7a5ffb5',
  'interval': '1 mon',
  'item_types': ['PSScene3Band', 'REOrthoTile'],
  'last_acquired': '2016-02-01T00:00:00.000Z',
  'level': 15,
  'name': 'global_monthly_2016_01_mosaic',
  'product_type': 'timelapse'},
 {'_links': {'_self': 'https://api.planet.com/basemaps/v1/mosaics/f2952136-69bc-4c31-bc21-a2e3919042a1?api_key=5a78a5a455644480a8a40ce5b4af9caa',
   'tiles': 'https://tiles.planet.com/basemaps/v1/planet-tiles/global_monthly_2016_12_mosai

In [15]:
for mosaic in result["mosaics"]:
    mosaics = db.mosaics
    mosaic_id = mosaics.insert_one(mosaic).inserted_id #Escribir en repositorio el link al nuevo tileresult

In [16]:
#load deforestation Points

def loadDefPoints(pointPath):#Falta especificar fecha o arvhivo en los argumentos.
    gdf=gpd.GeoDataFrame.from_file(pointPath)
    gdf["x"] = gdf.centroid.y
    gdf["y"] = gdf.centroid.x
    df=gdf[["x","y"]]
    return df

In [17]:
#Se calculan las cuatro esquinas del chip para el punto i
def chipVertices(p,delta,pixeles):
    delta_grados=delta*pixeles/2
    chipVertices=[]
    chipVertices.append((df["x"][i]+delta_grados,df["y"][i]+delta_grados))
    chipVertices.append((df["x"][i]-delta_grados,df["y"][i]-delta_grados))
    chipVertices.append((df["x"][i]+delta_grados,df["y"][i]-delta_grados))
    chipVertices.append((df["x"][i]-delta_grados,df["y"][i]+delta_grados))
    return chipVertices


In [18]:
#Codigo para optener el tile correspondiente a una coordenada. 
#Se identifica el tile para cada esquina.
def chipTiles(p,delta,pixeles,z):
    chipV=chipVertices(p,delta,pixeles)
    tiles=[]
    for p in chipV:
        latitude, longitude, zoom = p[0],p[1], z
        tile = Tile.for_latitude_longitude(latitude, longitude, zoom)
        tiles.append(tile.google)
    tiles = np.unique(tiles, axis=0) 
    zz = [] 
    for i in tiles:
       zz.append(tuple(i))
    tiles=zz
    return tiles

In [21]:
#Peticion de til 1024x1024 metros til segu sistema z,x,y para el mosaico disponible
def getTile(mosaicName,x,y,z,dirNameNoDef, fileName):
    #try:
        
    result = requests.get('https://tiles.planet.com/basemaps/v1/planet-tiles/%s/gmap/%s/%s/%s.png'%(mosaicName,str(z),x,y),
                              auth=HTTPBasicAuth(os.environ['PL_API_KEY_IDEAM'],''))#os.environ['PL_API_KEY_IDEAM'],''))
    with open(dirNameNoDef+'/'+fileName, 'wb') as f: 
        f.write(result.content) #Guardar imagen en archivo
        google_x, google_y, zoom = x, y, 15
        tile = Tile.from_google(google_x, google_y, zoom)
        post = {"dir_name":dirNameNoDef,
                    "file_name":fileName,
                    "mosaic_name": mosaicName,
                    "x": str(x),
                    "y": str(y),
                    "z": str(z),
                    "bounds": tile.bounds}
        #print(post)
        tiles = db.tiles
        tiles_id = tiles.insert_one(post).inserted_id #Escribir en repositorio el link al nuevo tile
    return "ok"
    #except:
        #return "error"
        
def getTiles(tiles,mosaicName,z,dirNameDef,dirNameNoDef):
    b=0
    c=0
    d=0
    for tile in tiles:
        fileName='%s_%s_%s_%s.png'%(mosaicName,z,tile[0],tile[1])
        #COnsultar si tile existe en BD Def; si si pasa si no descarga
        tiles = db.tiles
        q=tiles.find({"dir_name": dirNameDef,"file_name": fileName}).count()
        #Consutlar si el tile existe en BD no def
        q2=tiles.find({"dir_name": dirNameNoDef,"file_name": fileName}).count()
        if q == 0 and q2==0:
            a=getTile(mosaicName,tile[0],tile[1],z,dirNameNoDef, fileName) 
            if a=="ok":
                b=b+1
            else:
                c=c+1
        else:
            
            pass
        d=d+1
        #x= d%(round(tiles.count_documents()/10,0))
        #if x == 0:
              #print(x*100,'%')
    return (c,b)

In [23]:
df=loadDefPoints('porFinNoDefPoints.geojson')
a=0
b=df.shape[0]-1
df1=df[a:b] #Partir el dataset para poder paralelizar la funcion.
df1=df1.reset_index()
dirNameDef="global_monthly_2019_01_mosaic"#"global_monthly_2019_01_mosaic"
dirNameNoDef="global_monthly_2019_01_mosaic_NoDef"#"global_monthly_2019_01_mosaic_NoDef"

#Worker1 Descargar Tiles si ya no estan en BD y crear chips para cada punto para su subdataset asignado.

a1=0
b1=0
for i in range(df1.shape[0]):
    delta=0.0000429153086046707 #grados por pixel en tile planet xyz @z=15
    pixeles=256
    z=15
    p=(df["x"][i],df["y"][i])
    mosaicName="global_monthly_2019_01_mosaic" #Aqui esete nombre depende de la fecha de descarga
    tiles=chipTiles(p,delta,pixeles,z)
    g=getTiles(tiles,mosaicName,z,dirNameDef,dirNameNoDef)
    a1=a1+g[1]
    b1=b1+g[0]
    if i%100==0:
        print('Download at: ',(100*i)/df1.shape[0],'% ', a1, "tiles downloades of ", a1+b1)
        
clientMongo.close()
print(a1, "tiles downloades of ", a1+b1)
    


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


Download at:  0.0 %  0 tiles downloades of  0
Download at:  3.0543677458766036 %  228 tiles downloades of  228
Download at:  6.108735491753207 %  628 tiles downloades of  628
Download at:  9.16310323762981 %  1026 tiles downloades of  1026
Download at:  12.217470983506415 %  1411 tiles downloades of  1411
Download at:  15.271838729383017 %  1800 tiles downloades of  1800
Download at:  18.32620647525962 %  2186 tiles downloades of  2186
Download at:  21.380574221136225 %  2571 tiles downloades of  2571
Download at:  24.43494196701283 %  2958 tiles downloades of  2958
Download at:  27.489309712889433 %  3334 tiles downloades of  3334
Download at:  30.543677458766034 %  3716 tiles downloades of  3716
Download at:  33.59804520464264 %  4084 tiles downloades of  4084
Download at:  36.65241295051924 %  4470 tiles downloades of  4470
Download at:  39.706780696395846 %  4838 tiles downloades of  4838
Download at:  42.76114844227245 %  5208 tiles downloades of  5208
Download at:  45.81551618814